## Hierarchical Forecasting with Different level's of Data

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../')

import constants as const
import src.utils as util
import src.calculate_errors as err

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
def calculate_grid_error(model_path):
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    
    results_df = pd.read_csv(f'../{model_path}/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    if model_path == 'benchmark_results/arima':
        forecasts = results_df['average_fc'].values
    else:
        forecasts = results_df['fc'].values
    horizon = 14
    
    mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    return mean_err, error_dist

In [3]:
def read_loss_curve(path):
    return pd.read_pickle(f'../{path}/training_loss_grid_iteration')

def plot_loss(loss_array, models):

    fig = make_subplots(rows = len(loss_array), shared_xaxes=True)
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ), row = i+1, col=1)
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'), row = i+1, col=1)
        i +=1
        
    fig.update_xaxes(title="Epochs", row=i, col=1)
    fig.show()
    
def show_loss(path_list, model_names):
    loss_vals = []
    for path in path_list:
        loss_vals.append(read_loss_curve(path))
    plot_loss(loss_vals, model_names)

## Model 1 - Architecture

<img src="../images/New_images/model1.png">

In [4]:
path1 = 'combined_nn_results/refined_models/model1'
rmse1, dist1 = calculate_grid_error(path1)

In [5]:
rmse1

0.4689518104839537

## Model 2 - Architecture

<img src="../images/new_images/model2.png">

In [8]:
path2 = 'combined_nn_results/refined_models/model2'
rmse2, dist2 = calculate_grid_error(path2)

In [9]:
rmse2

0.4851028450736508

## Model 3 - Architecture

<img src="../images/new_images/model3.png">

In [10]:
# dense layer 14 output from each branch
path3 = 'combined_nn_results/refined_models/model3'
rmse3, dist3 = calculate_grid_error(path3)

In [11]:
rmse3

0.40207853214251055

## Model 4 - Architecture 

<img src="../images/new_images/model4.png">

In [12]:
path4 = 'combined_nn_results/refined_models/model4'
rmse4, dist4 = calculate_grid_error(path4)

In [13]:
rmse4

0.48270205794177873

## Model 5 - Architecture

<img src="../images/new_images/model5.png">

In [14]:
path5 = 'combined_nn_results/refined_models/model5'
rmse5, dist5 = calculate_grid_error(path5)

In [15]:
rmse5

0.4746874638641852

## Model 6

Same architecture as Model 5 but with new data

In [16]:
# model 6 used the new pre processing of data

path6 = 'combined_nn_results/refined_models/model6'
rmse6, dist6 = calculate_grid_error(path6)

In [17]:
rmse6

0.3899444423059909

## Model 7

Reduced the convolution layers to 2

In [18]:
path7 = 'combined_nn_results/refined_models/model7'
rmse7, dist7 = calculate_grid_error(path7)

In [19]:
rmse7

0.3390148849998616

## Model 8

In [21]:
path8 = 'combined_nn_results/refined_models/model8'
rmse8, dist8 = calculate_grid_error(path8)

In [22]:
rmse8

0.40559583690922857

## Model 9

In [38]:
path9 = 'combined_nn_results/refined_models/model9'
rmse9, dist9 = calculate_grid_error(path9)

In [39]:
rmse9

0.3552818973681426

In [57]:
path10 = 'combined_nn_results/refined_models/model10'
rmse10, dist10 = calculate_grid_error(path10)

In [58]:
rmse10

0.3641159126739144

In [59]:
path11 = 'combined_nn_results/refined_models/model11'
rmse11, dist11 = calculate_grid_error(path11)

In [60]:
rmse11

0.38913993761101306

## Grid only model

In [66]:
path_grid = 'combined_nn_results/refined_models/grid_model'
rmse_grid, dist_grid = calculate_grid_error(path_grid)

In [67]:
rmse_grid

0.40673213392218766

In [6]:
arimapath = 'benchmark_results/arima'
arima_gridrmse, arima_grid = calculate_grid_error(arimapath)

In [7]:
arima_gridrmse

0.3881505121536341

## Residual Model

In [77]:
res_model = 'combined_nn_results/refined_models/residual_model'
rmse_res, dist_res = calculate_grid_error(res_model)

In [78]:
rmse_res

0.41429881593906076

In [82]:
res_model2 = 'combined_nn_results/refined_models/residual_model2'
rmse_res2, dist_res2 = calculate_grid_error(res_model2)
rmse_res2

0.36800180112890035

In [86]:
res_model3 = 'combined_nn_results/refined_models/residual_model3'
rmse_res3, dist_res3= calculate_grid_error(res_model3)
rmse_res3

0.37354493435902414

Postcode only

In [87]:
pc_only = 'combined_nn_results/refined_models/postcode_only'
rmse_pc, dist_pc= calculate_grid_error(pc_only)
rmse_pc

0.341281379861684

In [99]:
pc_only2 = 'combined_nn_results/refined_models/postcode_only2' # without layer normalization
rmse_pc2, dist_pc2= calculate_grid_error(pc_only2)
rmse_pc2

0.42601355474826996

Residual block

In [6]:
res_block = 'combined_nn_results/refined_models/residual_block'
rmse_resblock, dist_resblock= calculate_grid_error(res_block)
rmse_resblock

0.3894877543909436

In [7]:
res_block2 = 'combined_nn_results/refined_models/residual_block2'
rmse_resblock2, dist_resblock2= calculate_grid_error(res_block2)
rmse_resblock2

0.3808541439075752

## Loss Curves

In [83]:
# show_loss([path1, path2, path3, path4, path5, path6, path7], ['model 1', 'model 2', 'model 3', 'model 4', 'model 5', 'model 6', 'model 7'])

show_loss([path6, path7, path8, path9, path11, path_grid, res_model, res_model2], ['model 6', 'model 7', 'model 8', 'model 9', 'model 11', 'grid', 'res', 'res2'])


## Plot the forecasts

In [43]:
def change_df(dataframe):
    dataframe.index = pd.to_datetime(dataframe.reset_index()['date_str'])
    dataframe = dataframe.resample("1H").mean()
    return dataframe

In [44]:
arima = pd.read_csv(f'../benchmark_results/arima/grid.csv', index_col=[0])
arima = change_df(arima)

In [45]:
tbats = pd.read_csv(f'../benchmark_results/tbats/grid.csv', index_col=[0])
tbats = change_df(tbats)

In [46]:
model1_fc = pd.read_csv(f'../{path1}/grid.csv', index_col=[0])
model1_fc = change_df(model1_fc)

In [47]:
model2_fc = pd.read_csv(f'../{path2}/grid.csv', index_col=[0])
model2_fc = change_df(model2_fc)

In [48]:
model3_fc = pd.read_csv(f'../{path3}/grid.csv', index_col=[0])
model3_fc = change_df(model3_fc)

In [49]:
model4_fc = pd.read_csv(f'../{path4}/grid.csv', index_col=[0])
model4_fc = change_df(model4_fc)

In [50]:
model5_fc = pd.read_csv(f'../{path5}/grid.csv', index_col=[0])
model5_fc = change_df(model5_fc)

In [51]:
path_cnn = 'cnn_results/tcn_new/dilation_2/final_results'

cnn_uni = pd.read_csv(f'../{path_cnn}/grid.csv', index_col=[0])
cnn_uni = change_df(cnn_uni)

In [52]:
model6_fc = pd.read_csv(f'../{path6}/grid.csv', index_col=[0])
model6_fc = change_df(model6_fc)

In [53]:
model7_fc = pd.read_csv(f'../{path7}/grid.csv', index_col=[0])
model7_fc = change_df(model7_fc)

In [54]:
model8_fc = pd.read_csv(f'../{path8}/grid.csv', index_col=[0])
model8_fc = change_df(model8_fc)

In [55]:
model9_fc = pd.read_csv(f'../{path9}/grid.csv', index_col=[0])
model9_fc = change_df(model9_fc)

In [62]:
model11_fc = pd.read_csv(f'../{path11}/grid.csv', index_col=[0])
model11_fc = change_df(model11_fc)

In [69]:
modelgrid_fc = pd.read_csv(f'../{path_grid}/grid.csv', index_col=[0])
modelgrid_fc = change_df(modelgrid_fc)

In [80]:
modelres = pd.read_csv(f'../{res_model}/grid.csv', index_col=[0])
modelres = change_df(modelres)

In [84]:
modelres2 = pd.read_csv(f'../{res_model2}/grid.csv', index_col=[0])
modelres2 = change_df(modelres2)

In [91]:
modelresblock = pd.read_csv(f'../{res_block}/grid.csv', index_col=[0])
modelresblock = change_df(modelresblock)

In [93]:
modelresblock2 = pd.read_csv(f'../{res_block2}/grid.csv', index_col=[0])
modelresblock2 = change_df(modelresblock2)

In [94]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = model1_fc.index, y = model1_fc['power'], name = 'target generation'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = arima['average_fc'], name = 'arima'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model1_fc['fc'], name = 'Combine Grid+PC CNN forecasts'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model3_fc['fc'], name = 'Combine PC NN forecasts'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model4_fc['fc'], name = 'Combine Grid+PC NN forecasts'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model5_fc['fc'], name = 'Combine PC NN Dense forecasts'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model6_fc['fc'], name = 'new features'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = model9_fc['fc'], name = 'Removing Grid + Local Conv Stage'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = model11_fc['fc'], name = 'Full Pipeline'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = modelgrid_fc['fc'], name = 'Grid Only'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = modelres['fc'], name = 'Residual'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = modelres2['fc'], name = 'Residual2'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = modelresblock['fc'], name = 'Res block Fully'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = modelresblock2['fc'], name = 'Res block Conv'))





# fig.add_trace(go.Scatter(x = model1_fc.index, y = model8_fc['fc'], name = 'new features less cnn with relu'))
# fig.add_trace(go.Scatter(x = model1_fc.index, y = model9_fc['fc'], name = 'model 9'))



fig.update_yaxes(title = "Generation (kW)")
fig.show()
# fig.write_html('forecasts.html')

## Visualize Network Architecture

Moving Visulaisation to Colab

In [8]:
from keras.utils.vis_utils import plot_model
import tensorflow as tf

In [9]:
full_pipeline = tf.keras.models.load_model('../combined_nn_results/spartan_folders/saved_models/model11')

In [10]:
full_pipeline.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_postcode_6010 (InputLayer [(None, 14, 14)]     0                                            
__________________________________________________________________________________________________
input_postcode_6014 (InputLayer [(None, 14, 14)]     0                                            
__________________________________________________________________________________________________
input_postcode_6011 (InputLayer [(None, 14, 14)]     0                                            
__________________________________________________________________________________________________
input_postcode_6280 (InputLayer [(None, 14, 14)]     0                                            
______________________________________________________________________________________

In [12]:
plot_model(full_pipeline, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
